In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [6]:
data = pd.read_csv("data/Sentiment.csv")

In [10]:
data["text"].iloc[1]

"RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…"

In [13]:
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


#### X = data["text"] 
#### Target = data["sentiment"]  
#### Y = [1,2,3] with : 1 = Negatif, 2 = Neutral, 3 = Positive

In [43]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

#print(data[data['sentiment'] == 'Positive'].size)
#print(data[data['sentiment'] == 'Negative'].size)
#print(data[data["sentiment"] == "Neutral"].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #texts_to_sequences allows to convert text to integers 
# this is give us list of intergs [ [sentence1 : index_word1 index_word2 index_word3 index_word4]  [sentence2 : index_word1 index_word2] ] where each sentence has 
# differentes size and each word is number's representation. 
# To fix this size issue, we can give each sentence list, same size by uising pad_sequences function 

X = pad_sequences(X) #pad_sequences convert list to numpy array

#this give us [ [sentence1 : index_word1 index_word2 index_word3 index_word4]  [sentence2 : index_word1 index_word2 0 0] ]
#where 0 complete empty value

In [45]:
#print(tokenizer.word_counts)
#print(tokenizer.word_docs)
#print(tokenizer.word_index)
#print(tokenizer.document_count)

## LSTM Model 

Next, I compose the LSTM Network. Note that embed_dim, lstm_out, batch_size, droupout_x variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [49]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


Hereby I declare the train and test dataset.

In [50]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 28) (9293, 3)
(4578, 28) (4578, 3)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [51]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 30s - loss: 0.8422 - acc: 0.6392
Epoch 2/7
 - 28s - loss: 0.7074 - acc: 0.6985
Epoch 3/7
 - 28s - loss: 0.6483 - acc: 0.7269
Epoch 4/7
 - 28s - loss: 0.6082 - acc: 0.7444
Epoch 5/7
 - 28s - loss: 0.5674 - acc: 0.7657
Epoch 6/7
 - 28s - loss: 0.5431 - acc: 0.7769
Epoch 7/7
 - 28s - loss: 0.5197 - acc: 0.7869


Extracting a validation set, and measuring score and accuracy.

In [52]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.82
acc: 0.67


Test new tweet

In [53]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  204  724    5  130    6   55 1101   55   48    5  130]]
negative
